In [1]:
%matplotlib auto
import matplotlib.pyplot as plt
import cv2
import glob
import numpy as np
import pickle


Using matplotlib backend: TkAgg


In [2]:
images_folder = '/home/sykes/absluate_tracker/data/tracker_calibration_images/camera1/*'
images_names = sorted(glob.glob(images_folder))
images = []
for imname in images_names:
    im = cv2.imread(imname, 1)
    images.append(im)    

chessboardSize = (5,8)
frameSize = (1920,1080)


In [3]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 50
objp = objp * size_of_chessboard_squares_mm

In [4]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
for img in images:

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(1000)


cv2.destroyAllWindows()

In [5]:
np.array(objpoints).shape

(15, 40, 3)

In [6]:
np.array(imgpoints).shape

(15, 40, 1, 2)

In [8]:
imgpoints[0]

array([[[ 980.98956,  346.1916 ]],

       [[1018.0544 ,  368.67   ]],

       [[1056.2894 ,  392.18884]],

       [[1095.8727 ,  416.31882]],

       [[1136.8252 ,  441.28934]],

       [[ 944.83484,  368.9001 ]],

       [[ 982.0056 ,  392.37885]],

       [[1020.2695 ,  416.46667]],

       [[1059.996  ,  441.4643 ]],

       [[1101.0292 ,  467.34396]],

       [[ 907.4883 ,  392.49307]],

       [[ 944.70325,  416.59308]],

       [[ 983.0803 ,  441.6033 ]],

       [[1022.66205,  467.4984 ]],

       [[1063.7106 ,  494.33002]],

       [[ 869.0688 ,  416.7777 ]],

       [[ 906.07855,  441.79947]],

       [[ 944.38684,  467.7484 ]],

       [[ 984.1406 ,  494.5361 ]],

       [[1025.348  ,  522.3407 ]],

       [[ 829.0181 ,  442.03604]],

       [[ 866.0934 ,  467.9026 ]],

       [[ 904.4796 ,  494.65686]],

       [[ 944.1896 ,  522.46   ]],

       [[ 985.37054,  551.3034 ]],

       [[ 787.65784,  468.1888 ]],

       [[ 824.6401 ,  494.87662]],

       [[ 862.94257,  522.61

In [18]:
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

In [93]:
image = cv2.imread('/home/sykes/absluate_tracker/data/tracker_calibration_images/camera1/image15.png')
b,g,image_r = cv2.split(image)
img = cv2.inRange(image_r,240,255)
contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


for c in contours:
   # calculate moments for each contour
   M = cv2.moments(c)

   # calculate x,y coordinate of center
   cX = int(M["m10"] / M["m00"])
   cY = int(M["m01"] / M["m00"])
   print('centroid: X:{}, Y:{}'.format(cX, cY)) 
   cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)
   cv2.putText(img, "centroid", (cX -25, cY -25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

   # display the image
   cv2.imshow("Image", img)
   cv2.waitKey(0)


centroid: X:743, Y:496


In [9]:
with open ("/home/sykes/absluate_tracker/data/data.pickle", 'rb') as f: 
    data = np.array(pickle.load(f))

In [12]:
objp = np.zeros((len(data),1, 3), np.float32)

In [14]:
objp.shape

(31, 1, 3)

In [15]:
imgpoints1 = []
imgpoints2 = []
for points in data:
    imgpoints1.append([[points[0]]])
    imgpoints2.append([[points[1]]])
imgpoints2 = np.array(imgpoints2,np.float32)
imgpoints1 = np.array(imgpoints1,np.float32)

In [21]:
imgpoints2.shape

(31, 1, 1, 2)

In [22]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)
R, T, E, F = cv2.stereoCalibrate(
    objp, imgpoints1, imgpoints2, 
    cameraMatrix, dist, cameraMatrix, dist,
      (1920, 1080), criteria = criteria, flags = cv2.CALIB_FIX_INTRINSIC)

error: OpenCV(4.9.0) /io/opencv/modules/calib3d/src/calibration.cpp:1088: error: (-215:Assertion failed) (count >= 4) || (count == 3 && useExtrinsicGuess) in function 'cvFindExtrinsicCameraParams2'


In [26]:
np.array(data[:,0], np.float32)

array([[ 977.,  611.],
       [ 843.,  513.],
       [ 686.,  618.],
       [ 671.,  534.],
       [ 801.,  452.],
       [ 896.,  519.],
       [ 894.,  465.],
       [ 951.,  427.],
       [ 878.,  379.],
       [ 800.,  428.],
       [ 793.,  358.],
       [ 613.,  237.],
       [ 756.,  166.],
       [ 878.,  105.],
       [1285.,  320.],
       [1011.,  517.],
       [ 674.,  290.],
       [ 471.,  398.],
       [ 434.,  264.],
       [ 620.,  125.],
       [ 736.,   72.],
       [ 938.,  183.],
       [ 941.,  449.],
       [1184.,  611.],
       [1026.,  742.],
       [ 660.,  461.],
       [ 933.,  298.],
       [ 931.,  184.],
       [1237.,  353.],
       [ 987.,  532.],
       [ 983.,  756.]], dtype=float32)

In [29]:
mat, mask = cv2.findFundamentalMat(np.array(data[:,0], np.float32),np.array(data[:,1], np.float32),cv2.FM_LMEDS,0,0)

In [30]:
mat

array([[-6.33799911e-09, -4.90657277e-07, -5.21027244e-04],
       [-5.11449030e-07,  1.58923539e-08, -6.45588452e-04],
       [-5.13480680e-04,  1.59287618e-03,  1.00000000e+00]])